In [38]:
!pip3 install git+https://github.com/SebMilardo/multiqueue

  Cloning https://github.com/SebMilardo/multiqueue to /private/var/folders/dd/8g47btbn3r51stt80t5hbhzr0000gn/T/pip-req-build-4j7r50_r
  Running command git clone -q https://github.com/SebMilardo/multiqueue /private/var/folders/dd/8g47btbn3r51stt80t5hbhzr0000gn/T/pip-req-build-4j7r50_r
  Stored in directory: /private/var/folders/dd/8g47btbn3r51stt80t5hbhzr0000gn/T/pip-ephem-wheel-cache-lojycq_g/wheels/3e/ad/21/eec2a884031f08f090a1a4e11d28e61699906f9058f72b463c
Successfully built multiqueue


In [41]:
from multiqueue import multiqueue

In [42]:
def a_test(x):
    print(x)
    time.sleep(4)
    return True, x*2

res = multiqueue(list(range(10)), function=a_test)

0
1
2


3
4
5
67

8
9


In [7]:
res = [a_test(x) for x in tqdm_notebook(range(10))]

In [37]:
res

[[0, 0],
 [1, 2],
 [2, 4],
 [3, 6],
 [4, 8],
 [5, 10],
 [6, 12],
 [7, 14],
 [8, 16],
 [9, 18]]